In [1]:
import cv2
import os
from tqdm import tqdm
import numpy as np

In [2]:
img_dir = 'images'
print(os.listdir(img_dir))

['Blue', 'Green', 'red']


In [3]:
dataset = []
for i in os.listdir(img_dir):
    path = os.path.join(img_dir,i)
#     print(path)
    for im in os.scandir(path):
        img = cv2.imread(im.path)
        img = cv2.resize(img,(224,224))
        dataset.append([img,i])

In [ ]:
import random
random.shuffle(dataset)

In [4]:
x = []
y = []
for features,labels in dataset:
    x.append(features)
    y.append(labels)

In [5]:
y[:3]

['Blue', 'Blue', 'Blue']

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
x = np.array(x)

In [8]:
x.shape

(397, 224, 224, 3)

In [9]:
y.shape

(397,)

In [10]:
y = y.reshape(y.shape[0],1)

In [11]:
y.shape

(397, 1)

In [12]:
x = x/255.0

In [13]:
x[0]

array([[[0.6       , 0.0745098 , 0.        ],
        [0.6       , 0.0745098 , 0.        ],
        [0.6       , 0.0745098 , 0.        ],
        ...,
        [0.69411765, 0.14901961, 0.05490196],
        [0.69411765, 0.14901961, 0.05490196],
        [0.69411765, 0.14901961, 0.05490196]],

       [[0.6       , 0.0745098 , 0.        ],
        [0.6       , 0.0745098 , 0.        ],
        [0.6       , 0.0745098 , 0.        ],
        ...,
        [0.69411765, 0.14901961, 0.05490196],
        [0.69411765, 0.14901961, 0.05490196],
        [0.69411765, 0.14901961, 0.05490196]],

       [[0.6       , 0.0745098 , 0.        ],
        [0.6       , 0.0745098 , 0.        ],
        [0.6       , 0.0745098 , 0.        ],
        ...,
        [0.69411765, 0.14901961, 0.05490196],
        [0.69411765, 0.14901961, 0.05490196],
        [0.69411765, 0.14901961, 0.05490196]],

       ...,

       [[0.6       , 0.0745098 , 0.        ],
        [0.6       , 0.0745098 , 0.        ],
        [0.6       , 0

In [14]:
x.shape

(397, 224, 224, 3)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

In [16]:
model = Sequential([Conv2D(256,(3,3),input_shape=(224,224,3),activation='relu'),
                    MaxPooling2D((2,2)),
                    BatchNormalization(),
                    Dropout(0.2),
                    Conv2D(256,(3,3),activation='relu'),
                    MaxPooling2D((2,2)),
                    BatchNormalization(),
                    Dropout(0.2),
                    Conv2D(256,(3,3),activation='relu'),
                    MaxPooling2D((2,2)),
                    BatchNormalization(),
                    Dropout(0.2),
                    Flatten(),
                    Dense(64,activation='relu'),
                    Dense(3,activation='softmax')
                   ])

In [17]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x,y,batch_size=32,epochs=10)

Train on 397 samples
Epoch 1/10
397/397 [==============================] - 105s 265ms/sample - loss: 16.0633 - accuracy: 0.8312
Epoch 2/10
397/397 [==============================] - 106s 266ms/sample - loss: 9.9920 - accuracy: 0.9345
Epoch 3/10
397/397 [==============================] - 106s 267ms/sample - loss: 3.6432 - accuracy: 0.9673
Epoch 4/10
397/397 [==============================] - 106s 268ms/sample - loss: 5.3537 - accuracy: 0.9723
Epoch 5/10
397/397 [==============================] - 108s 271ms/sample - loss: 0.1796 - accuracy: 0.9748
Epoch 6/10
397/397 [==============================] - 109s 276ms/sample - loss: 1.1406 - accuracy: 0.9496
Epoch 7/10
397/397 [==============================] - 110s 276ms/sample - loss: 0.4240 - accuracy: 0.9547
Epoch 8/10
397/397 [==============================] - 111s 278ms/sample - loss: 1.1388 - accuracy: 0.9849
Epoch 9/10
397/397 [==============================] - 111s 279ms/sample - loss: 4.9850 - accuracy: 0.9547
Epoch 10/10
320/397 [===

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 256)     7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 256)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 111, 111, 256)     1024      
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 256)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 256)     590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 54, 54, 256)       1

In [24]:
model.save('trained_model.h5')